# 工具包导入&数据读取
## 工具包导入

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
import gc
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline 

/home/user/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%pwd

u'/home/user/secury/scr'

## 数据读取
- 为了方便分析，我们读取3000万条数据进行处理

In [3]:
# path = '../final_input/'
# train = pd.read_csv(path + 'final_train.csv',nrows=1000000)
# test = pd.read_csv(path + 'final_test.csv',nrows=1000000)

In [4]:
path = '../input/'
train = pd.read_csv(path + 'final_train.csv')
#test = pd.read_csv(path + 'final_test.csv')

# 特征工程 & 验证结果(1-Gram)

In [5]:
train_data = train[['file_id','label']].drop_duplicates()
train_data.head()

,file_id,label
0,1,5
6786,2,2
7602,3,0
8065,4,0
10111,5,0


In [6]:
train_data['label'].value_counts()

0    4978
5    4289
7    1487
2    1196
3     820
6     515
1     502
4     100
Name: label, dtype: int64

## 全局特征:
- File_id (Api): count,nunique
- File_id (Tid): count,nunique,max,min,quantile(20,40,50,60,80),std,range
- File_id (Return Value): count,nunique,max,min,quantile(20,40,50,60,80),std,range
- File_id (Index): count,nunique,max,min,quantile(20,40,50,60,80),std,range

### File_id (Api): count,nunique

In [7]:
train.head()

,file_id,label,api,tid,index
0,1,5,LdrLoadDll,2488,0
1,1,5,LdrGetProcedureAddress,2488,1
2,1,5,LdrGetProcedureAddress,2488,2
3,1,5,LdrGetProcedureAddress,2488,3
4,1,5,LdrGetProcedureAddress,2488,4


In [8]:
api_opt = ['count','nunique'] 
for opt in api_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['api'].agg({'fileid_api_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id')  

count


/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique


In [9]:
train_data.head()

,file_id,label,fileid_api_count,fileid_api_nunique
0,1,5,6786,116
1,2,2,816,30
2,3,0,463,42
3,4,0,2046,51
4,5,0,10002,65


### File_id (Tid): count,nunique,max,min,quantile(20,40,50,60,80),std,range

In [10]:
tid_opt = ['count','nunique','max','min','median','std'] 
for opt in tid_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['tid'].agg({'fileid_tid_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id') 

count


/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique
max
min
median
std


In [11]:
secs = [0.2,0.4,0.6,0.8]
for sec in secs: 
    train_data['fileid_tid_quantile_' + str(sec * 100)] = train.groupby(['file_id'])['tid'].quantile(sec).values
 
train_data['fileid_tid_range'] = train.groupby(['file_id'])['tid'].quantile(0.975).values - train.groupby(['file_id'])['tid'].quantile(0.0125).values

### File_id (Index): count,nunique,max,min,quantile(20,40,50,60,80),std,range

In [12]:
index_opt = ['count','nunique','max','min','median','std'] 
for opt in index_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['index'].agg({'fileid_index_' + opt: opt}).reset_index() 
    train_data = pd.merge(train_data,tmp,how='left', on='file_id') 

count


/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique
max
min
median
std


In [13]:
secs = [0.2,0.4,0.6,0.8]
for sec in secs: 
    train_data['fileid_index_quantile_' + str(sec * 100)] = train.groupby(['file_id'])['index'].quantile(sec).values
 
train_data['fileid_index_range'] = train.groupby(['file_id'])['index'].quantile(0.975).values - train.groupby(['file_id'])['index'].quantile(0.0125).values

### 全局特征的线下验证 <font color=red>( 0.0969482)</font>

#### 评估指标

In [14]:
def lgb_logloss(preds,data):
    labels_ = data.get_label()
    classes_ = np.unique(labels_) 
    preds_prob = []
    for i in range(len(classes_)):
        preds_prob.append(preds[i*len(labels_):(i+1) * len(labels_)])
    preds_prob_ = np.vstack(preds_prob) 
    
    loss = [] 
    for i in range(preds_prob_.shape[1]):  # 样本个数
        sum_ = 0  
        for j in range(preds_prob_.shape[0]): #类别个数
            pred = preds_prob_[j,i] # 第i个样本预测为第j类的概率
            if  j == labels_[i]:
                sum_ += np.log(pred)
            else:
                sum_ += np.log(1 - pred) 
             
        loss.append(sum_)  
         
    return 'loss is: ' ,-1 * (np.sum(loss) / preds_prob_.shape[1]),False

#### 训练特征 & 标签

In [15]:
train_features = [col for col in train_data.columns if col!='label' and col!='file_id']
train_label = 'label'

In [16]:
train_X, test_X, train_Y, test_Y = train_test_split( train_data[train_features],train_data[train_label].values, test_size = 0.33) 
del _
gc.collect()

train_ind = train_X.index
test_ind = test_X.index

In [17]:
# dtrain = lgb.Dataset(train_X,train_Y) 
# dval   = lgb.Dataset(test_X,test_Y, reference = dtrain) 

# params = {
#         'task':'train', 
#         'num_leaves': 255,
#         'objective': 'multiclass',
#         'num_class':8,
#         #'min_data_in_leaf': 40,
#         'min_data_in_leaf': 1,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.85,
#         'bagging_fraction': 0.9,
#         'bagging_freq': 5, 
#         'max_bin':128,
#         'num_threads': 10,
#         'random_state':100
#     }  
# lgb_model_0_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

### 全局特征扩充
- File_id + return_value分段：计数

## 局部组合特征(展开形式)
### File_id + Api  
- File_id + Api (tid): count,nunique
- File_id + Api (return value): nunique, max, min, median, std
- File_id + Api (index):  nunique, max, min, median, std



#### File_id + Api (tid): count,nunique

In [18]:
def groupby_pivot_features(data_merge, data_orig , groupby_features,col1 = None, col2 = None, opts = None):
    for opt in opts:
        print(opt)
        train_split = data_orig.groupby(['file_id',col1])[col2].agg({'fileid_' + col1 + '_'+col2+'_'+ str(opt):opt}).reset_index() 
        
        train_split_ =  pd.pivot_table(train_split, values = 'fileid_' + col1 + '_'+col2+'_'+ str(opt), index=['file_id'],columns=[col1])
        new_cols = [ 'fileid_' + col1 + '_'+col2+  '_' + opt + '_' + str(col) for col in train_split_.columns]
        
        groupby_features.append(new_cols)
        train_split_.columns = new_cols 

        train_split_.reset_index(inplace = True)
        
        data_merge = pd.merge(data_merge,train_split_,how='left', on='file_id') 
    return data_merge,groupby_features 
    

In [19]:
groupby_features = []
api_opts = ['count', 'nunique']
train_data_,groupby_features = groupby_pivot_features(train_data, train, groupby_features, col1 = 'api', col2 = 'tid', opts = api_opts)

count


/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique


#### File_id + Api (return value): nunique, max, min, median, std

In [20]:
# api_opts = ['nunique','max','min','median','std']
# train_data_,groupby_features = groupby_pivot_features(train_data_, train, groupby_features, col1 = 'api', col2 = 'return_value', opts = api_opts) 

####  File_id + Api(index): nunique, max, min, median, std

In [21]:
api_opts = ['nunique','max','min','median','std']
train_data_,groupby_features = groupby_pivot_features(train_data_, train, groupby_features, col1 = 'api', col2 = 'index', opts = api_opts) 

nunique


/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


max
min
median
std


In [22]:
train_data_.head()

,file_id,label,fileid_api_count,fileid_api_nunique,fileid_tid_count,fileid_tid_nunique,fileid_tid_max,fileid_tid_min,fileid_tid_median,fileid_tid_std,...,fileid_api_index_std_recv,fileid_api_index_std_recvfrom,fileid_api_index_std_select,fileid_api_index_std_send,fileid_api_index_std_sendto,fileid_api_index_std_setsockopt,fileid_api_index_std_shutdown,fileid_api_index_std_socket,fileid_api_index_std_system,fileid_api_index_std_timeGetTime
0,1,5,6786,116,6786,11,2812,2488,2488,83.881299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.185986,NaN,NaN
1,2,2,816,30,816,4,2604,2320,2480,101.506783,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.055816,NaN,NaN
2,3,0,463,42,463,1,2208,2208,2208,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0,2046,51,2046,9,2980,2284,2340,150.460506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0,10002,65,10002,6,2676,2500,2596,49.556301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1阶特征的线下验证(File_id + Api)（<font color=red>0.0347293</font>）

### File_id + Index  
- File_id + Index (api): count,nunique
- File_id + Index (return value): nunique, max, min, median, std(暂时先搁置)
- File_id + Index (tid):  nunique, max, min, median, std(暂时先搁置)


#### File_id +Tid (api): count,nunique

#### File_id + Index特征过拟合，删除


In [23]:
# delcol = []
# for i in range(2):
#     for item in groupby_features2[i]:
#         delcol.append(item)

In [24]:
# train_data_.drop(delcol,axis=1,inplace=True)

## 特征补充（加入index的差值特征）
- File_id + Api (index_diff): 'nunique','max','min','median','std'

In [25]:
train_diff = train.groupby(['file_id','tid'])['index'].diff().fillna(-999).values

In [26]:
train['index_diff'] = train_diff

In [27]:
train_diff = train.loc[train.index_diff!=-999] 

In [28]:
api_opts = ['nunique','max','min','median','std']
train_data_,groupby_features = groupby_pivot_features(train_data_, train_diff, groupby_features, col1 = 'api', col2 = 'index_diff', opts = api_opts) 

nunique


/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


max
min
median
std


### 线下验证(<font color=red>0.0346954</font>)

In [29]:
# train_features = [col for col in train_data_.columns if col!='label' and col!='file_id']
# train_label = 'label'
# print(len(train_features))
# dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
# dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

# params = {
#         'task':'train', 
#         'num_leaves': 255,
#         'objective': 'multiclass',
#         'num_class':6,
#         'min_data_in_leaf': 40,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.85,
#         'bagging_fraction': 0.9,
#         'bagging_freq': 5, 
#         'max_bin':128,
#         'num_threads': 64,
#         'random_state':100
#     }  
# lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

### 删除quantile,std统计变量之后的验证(<font color=red>0.0350054</font>)  

In [30]:
# train_features = [col for col in train_data_.columns if col!='label' and col!='file_id' and 'std' not in col and 'quantile' not in col]
# train_label = 'label'
# print(len(train_features))
# dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
# dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

# params = {
#         'task':'train', 
#         'num_leaves': 255,
#         'objective': 'multiclass',
#         'num_class':6,
#         'min_data_in_leaf': 40,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.85,
#         'bagging_fraction': 0.9,
#         'bagging_freq': 5, 
#         'max_bin':128,
#         'num_threads': 64,
#         'random_state':100
#     }  
# lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

In [31]:
# train_data_.to_csv('/data/Data_JieZhang/TC_SAFE/train_val/train_data.csv',index = None) 

# 特征工程& 验证结果 2-Gram
## 全局特征
### File_id（Api_2）:count,nunique

In [32]:
train['api_shift'] = train['api'].shift(-1)
train['api_2'] = train['api'] +'_' + train['api_shift']

In [33]:
train.drop(['api_shift'],axis=1,inplace=True)

In [34]:
api_count = train['api_2'].value_counts()

In [35]:
api_opt = ['count','nunique'] 
for opt in api_opt:
    print(opt)
    tmp = train.groupby(['file_id'])['api_2'].agg({'fileid_api_2_' + opt: opt}).reset_index() 
    train_data_ = pd.merge(train_data_,tmp,how='left', on='file_id')  

count


/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


nunique


## 局部特征
### File_id + tid (Api_2): count特征

In [36]:
api_value_counts = pd.DataFrame(api_count).reset_index()
api_value_counts.columns = ['api_2','api_2_count']

In [37]:
train = pd.merge(train, api_value_counts, on ='api_2' , how='left')

In [38]:
api_opts = ['count']
groupby_features =  []
train_data_,groupby_features = groupby_pivot_features(train_data_, train.loc[train.api_2_count>=20], groupby_features, col1 = 'api_2', col2 = 'tid', opts = api_opts)

count


/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


### 线下验证(<font color=red> 0.0330886</font>)

In [39]:
# train_features = [col for col in train_data_.columns if col!='label' and col!='file_id']
# train_label = 'label'
# print(len(train_features))
# dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
# dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

# params = {
#         'task':'train', 
#         'num_leaves': 255,
#         'objective': 'multiclass',
#         'num_class':6,
#         'min_data_in_leaf': 40,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.85,
#         'bagging_fraction': 0.9,
#         'bagging_freq': 5, 
#         'max_bin':128,
#         'num_threads': 64,
#         'random_state':100
#     }  
# lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

### File_id + index (Api_2): max,min特征

In [40]:
train_features = [col for col in train_data_.columns if col!='label' and col!='file_id']
train_label = 'label'
print(len(train_features))
train_X, test_X, train_Y, test_Y = train_test_split( train_data_[train_features],train_data_[train_label].values, test_size = 0.33) 

dtrain = lgb.Dataset(train_X,train_Y) 
dval   = lgb.Dataset(test_X,test_Y, reference = dtrain) 

params = {
        'task':'train', 
        'num_leaves': 255,
        'objective': 'multiclass',
        'num_class':8,
        #'min_data_in_leaf': 40,
        'min_data_in_leaf': 10,
        'learning_rate': 0.05,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.9,
        'bagging_freq': 5, 
        'max_bin':128,
        'num_threads': 64,
        'random_state':100
    }  
lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

12236
[1]	training's multi_logloss: 1.90825	training's loss is: : 2.81617	valid_1's multi_logloss: 1.92056	valid_1's loss is: : 2.83041
Training until validation scores don't improve for 50 rounds.
[2]	training's multi_logloss: 1.7664	training's loss is: : 2.64826	valid_1's multi_logloss: 1.78989	valid_1's loss is: : 2.67577
[3]	training's multi_logloss: 1.64441	training's loss is: : 2.50066	valid_1's multi_logloss: 1.67822	valid_1's loss is: : 2.54072
[4]	training's multi_logloss: 1.53808	training's loss is: : 2.36933	valid_1's multi_logloss: 1.58129	valid_1's loss is: : 2.42106
[5]	training's multi_logloss: 1.44341	training's loss is: : 2.25009	valid_1's multi_logloss: 1.49486	valid_1's loss is: : 2.31237
[6]	training's multi_logloss: 1.35794	training's loss is: : 2.14047	valid_1's multi_logloss: 1.41736	valid_1's loss is: : 2.21313
[7]	training's multi_logloss: 1.28042	training's loss is: : 2.03927	valid_1's multi_logloss: 1.3472	valid_1's loss is: : 2.12168
[8]	training's multi_log

[63]	training's multi_logloss: 0.109875	training's loss is: : 0.212393	valid_1's multi_logloss: 0.354111	valid_1's loss is: : 0.599118
[64]	training's multi_logloss: 0.105601	training's loss is: : 0.204351	valid_1's multi_logloss: 0.351429	valid_1's loss is: : 0.594374
[65]	training's multi_logloss: 0.101489	training's loss is: : 0.196593	valid_1's multi_logloss: 0.348967	valid_1's loss is: : 0.590063
[66]	training's multi_logloss: 0.0975043	training's loss is: : 0.189071	valid_1's multi_logloss: 0.346486	valid_1's loss is: : 0.585778
[67]	training's multi_logloss: 0.0936928	training's loss is: : 0.181861	valid_1's multi_logloss: 0.344046	valid_1's loss is: : 0.581524
[68]	training's multi_logloss: 0.0900635	training's loss is: : 0.17498	valid_1's multi_logloss: 0.341734	valid_1's loss is: : 0.577578
[69]	training's multi_logloss: 0.086594	training's loss is: : 0.16839	valid_1's multi_logloss: 0.33952	valid_1's loss is: : 0.573784
[70]	training's multi_logloss: 0.0833025	training's los

[123]	training's multi_logloss: 0.0108364	training's loss is: : 0.0215714	valid_1's multi_logloss: 0.326212	valid_1's loss is: : 0.561192
[124]	training's multi_logloss: 0.0104299	training's loss is: : 0.0207655	valid_1's multi_logloss: 0.326855	valid_1's loss is: : 0.56255
[125]	training's multi_logloss: 0.0100383	training's loss is: : 0.0199888	valid_1's multi_logloss: 0.327589	valid_1's loss is: : 0.564102
[126]	training's multi_logloss: 0.00966387	training's loss is: : 0.0192463	valid_1's multi_logloss: 0.328251	valid_1's loss is: : 0.565496
[127]	training's multi_logloss: 0.00930306	training's loss is: : 0.0185304	valid_1's multi_logloss: 0.329061	valid_1's loss is: : 0.567172
[128]	training's multi_logloss: 0.00895435	training's loss is: : 0.0178384	valid_1's multi_logloss: 0.329773	valid_1's loss is: : 0.568687
[129]	training's multi_logloss: 0.00862021	training's loss is: : 0.0171752	valid_1's multi_logloss: 0.330612	valid_1's loss is: : 0.570442
[130]	training's multi_logloss:

In [41]:
fea_imp = pd.DataFrame({'feature':train_features, 'imp':lgb_model_3_order.feature_importance()}).sort_values('imp')
important_features = fea_imp.loc[fea_imp.imp >=1, 'feature'].values
important_features = list(important_features)

important_features.append('file_id')
important_features.append('label')

In [42]:
# train_data_[important_features].to_csv('../feature_final/train_data_2gram.csv',index = None)

train_ind = train_X.index
test_ind = test_X.index

In [43]:
# len(important_features)

In [44]:
api_opts = ['max','min']
train_data_,groupby_features = groupby_pivot_features(train_data_[important_features], train.loc[train.api_2_count>=20], groupby_features, col1 = 'api_2', col2 = 'index', opts = api_opts)

max


/home/user/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


min


In [45]:
train_features = [col for col in train_data_.columns if col!='label' and col!='file_id' and 'std' not in col and 'quantile' not in col]
train_label = 'label'

train_ind = train_X.index
test_ind = test_X.index

dtrain = lgb.Dataset(train_data_.loc[train_ind,train_features],train_data_.loc[train_ind,train_label].values) 
dval   = lgb.Dataset(train_data_.loc[test_ind,train_features],train_data_.loc[test_ind,train_label].values, reference = dtrain) 

params = {
        'task':'train', 
        'num_leaves': 255,
        'objective': 'multiclass',
        'num_class':8,
        'min_data_in_leaf': 10,
        #'min_data_in_leaf': 1,
        'learning_rate': 0.05,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.9,
        'bagging_freq': 5, 
        'max_bin':128,
        'num_threads': 64,
        'random_state':100
    }  
lgb_model_3_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50, feval=lgb_logloss)  

[1]	training's multi_logloss: 1.90856	training's loss is: : 2.81654	valid_1's multi_logloss: 1.92087	valid_1's loss is: : 2.83077
Training until validation scores don't improve for 50 rounds.
[2]	training's multi_logloss: 1.76571	training's loss is: : 2.64745	valid_1's multi_logloss: 1.78921	valid_1's loss is: : 2.67497
[3]	training's multi_logloss: 1.64335	training's loss is: : 2.49943	valid_1's multi_logloss: 1.67637	valid_1's loss is: : 2.53858
[4]	training's multi_logloss: 1.53597	training's loss is: : 2.3668	valid_1's multi_logloss: 1.5782	valid_1's loss is: : 2.41745
[5]	training's multi_logloss: 1.44075	training's loss is: : 2.24685	valid_1's multi_logloss: 1.49107	valid_1's loss is: : 2.30783
[6]	training's multi_logloss: 1.3547	training's loss is: : 2.13643	valid_1's multi_logloss: 1.41293	valid_1's loss is: : 2.20775
[7]	training's multi_logloss: 1.27698	training's loss is: : 2.03491	valid_1's multi_logloss: 1.34204	valid_1's loss is: : 2.11535
[8]	training's multi_logloss: 1

[63]	training's multi_logloss: 0.109227	training's loss is: : 0.21106	valid_1's multi_logloss: 0.348992	valid_1's loss is: : 0.590537
[64]	training's multi_logloss: 0.104992	training's loss is: : 0.203088	valid_1's multi_logloss: 0.346622	valid_1's loss is: : 0.586319
[65]	training's multi_logloss: 0.100929	training's loss is: : 0.195426	valid_1's multi_logloss: 0.344024	valid_1's loss is: : 0.581803
[66]	training's multi_logloss: 0.0969796	training's loss is: : 0.18797	valid_1's multi_logloss: 0.341403	valid_1's loss is: : 0.577185
[67]	training's multi_logloss: 0.0931925	training's loss is: : 0.180804	valid_1's multi_logloss: 0.338987	valid_1's loss is: : 0.572995
[68]	training's multi_logloss: 0.0896026	training's loss is: : 0.173997	valid_1's multi_logloss: 0.336951	valid_1's loss is: : 0.569401
[69]	training's multi_logloss: 0.0861973	training's loss is: : 0.167526	valid_1's multi_logloss: 0.334829	valid_1's loss is: : 0.56571
[70]	training's multi_logloss: 0.0829002	training's lo

[123]	training's multi_logloss: 0.0108377	training's loss is: : 0.0215677	valid_1's multi_logloss: 0.323428	valid_1's loss is: : 0.55709
[124]	training's multi_logloss: 0.0104344	training's loss is: : 0.0207686	valid_1's multi_logloss: 0.324015	valid_1's loss is: : 0.558444
[125]	training's multi_logloss: 0.0100475	training's loss is: : 0.0200016	valid_1's multi_logloss: 0.324785	valid_1's loss is: : 0.560056
[126]	training's multi_logloss: 0.00967195	training's loss is: : 0.019257	valid_1's multi_logloss: 0.325642	valid_1's loss is: : 0.561782
[127]	training's multi_logloss: 0.00930444	training's loss is: : 0.0185283	valid_1's multi_logloss: 0.326291	valid_1's loss is: : 0.563197
[128]	training's multi_logloss: 0.00895312	training's loss is: : 0.0178314	valid_1's multi_logloss: 0.326888	valid_1's loss is: : 0.564504
[129]	training's multi_logloss: 0.0086164	training's loss is: : 0.0171633	valid_1's multi_logloss: 0.327696	valid_1's loss is: : 0.566191
[130]	training's multi_logloss: 0

In [46]:
fea_imp = pd.DataFrame({'feature':train_features, 'imp':lgb_model_3_order.feature_importance()}).sort_values('imp')
important_features = fea_imp.loc[fea_imp.imp >=1, 'feature'].values
important_features = list(important_features)

important_features.append('file_id')
important_features.append('label')

train_data_[important_features].to_csv('./train_data_2gram.csv',index = None)
 

# 附录
tf-idf的1Gram特征可以替换api的次数特征等，加入tf-idf有提升，提升较小